In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly
import plotly.graph_objects as go
%run plottingCPT.py

In [ ]:
dfBOOL2 = pd.read_csv('P17_optimization.csv',header=0,usecols = [1,2,3,4,5,6])
dfBOOL3 = pd.read_csv('P18_optimization.csv',header=0,usecols = [1,2,3,4,5,6])

In [ ]:
dfBOOL = pd.read_csv('P16_optimization.csv',header=0,usecols = [1,2,3,4,5,6])

In [ ]:
#DataFrame - Add Elevation range and Manipulate Columns
dfBOOL['ElevRange'] = dfBOOL['3'].astype(str) + '-' + dfBOOL['4'].astype(str)
dfBOOL.drop(columns = ['3','4'],inplace=True)
dfBOOL.columns = ['CPT_ID','TargetThickness','Threshold', 'Pass/Fail','ElevRange']
dfBOOL.set_index(['CPT_ID','TargetThickness','Threshold','ElevRange'],inplace=True)

#Groupby Operations  
dfBOOL_GRP = dfBOOL.groupby(level = [1,2,3])

#Get percentage of CPTs satisfying the criterion
df = dfBOOL_GRP.agg(lambda x : x.sum()/len(x))

## All Data - P16

In [ ]:
#df = df.loc[ (df > 0.25).values]
df.reset_index(level = [0,1,2],inplace=True)
df.sort_values(by = ['ElevRange','Threshold','TargetThickness'] ,  ascending= True,  inplace=True)
df.sort_values(by = ['ElevRange'] , ascending= True,  inplace=True,  key = lambda col : col.map(lambda x : float(x.split('-')[0])))

## All Data - P16

In [ ]:
for ii in np.sort(df['TargetThickness'].unique()):

    #dfPLOT = df.loc[(df['TargetThickness'] == ii) &  (df['Threshold'] <= 10)]
    dfPLOT = df.loc[df['TargetThickness'] == ii]
    fig = go.Figure(data=go.Heatmap(
                       z= dfPLOT['Pass/Fail']*100,
                       x= dfPLOT['Threshold'],
                       y= dfPLOT['ElevRange'], 
                       zmin = 0,
                       zmax = 100,
                       colorscale = plotly.colors.diverging.Portland,
                       colorbar_x = 1.05,

                        colorbar_title = dict(text              = 'Percentage of CPTs<br>Passing Criteria'),
                        colorbar_ticklabelposition = 'outside bottom' ))


    fig.update_layout(template = 'simple_white',font = dict(family = 'Arial', color = 'black'),
                       title_text = 'Weak Layer Heatmap for TT = ' + str(ii))
    fig.update_xaxes(tickmode = 'linear', dtick = 1, title = 'Tip Resistance Threshold Value (tsf)' )
    fig.update_yaxes(title = 'Elevation Range (ft)' )
    fig.show()

## Critical Scenarios

In [ ]:
cutArr = pd.cut(df['Threshold'], bins = [0,5,10,15], include_lowest = True)
def top(df, n=5, column='Pass/Fail'):
    return df.sort_values(by=column)[-n:]

df_GRP = df.groupby( by =['TargetThickness',cutArr])
#df_GRP.apply(top, n=2).head(50)
df_GRP_FILT = df_GRP.apply(top, n=2)
df_GRP_FILT.loc[df_GRP_FILT['Pass/Fail'] > 0.60]

In [ ]:
df_GRP_FILT.loc[df_GRP_FILT['Pass/Fail'] > 0.60]

## All Data - P16 [5-15]

In [ ]:
df.head()

In [ ]:
for ii in np.sort(df['TargetThickness'].unique()):

    dfPLOT = df.loc[df['TargetThickness'] == ii]
    fig = go.Figure(data=go.Heatmap(
                       z= dfPLOT['Pass/Fail']*100,
                       x= dfPLOT['Threshold'],
                       y= dfPLOT['ElevRange'], 
                       zmin = 0,
                       zmax = 100,
                       colorscale = plotly.colors.diverging.Portland,
                       colorbar_x = 1.05,

                        colorbar_title = dict(text              = 'Percentage of CPTs<br>Passing Criteria'),
                        colorbar_ticklabelposition = 'outside bottom' ))


    fig.update_layout(template = 'simple_white',font = dict(family = 'Arial', color = 'black'),
                       title_text = 'Weak Layer Heatmap for TT = ' + str(ii))
    fig.update_xaxes(tickmode = 'linear', dtick = 1, title = 'Tip Resistance Threshold Value (tsf)',
                    range = [5,15])
    fig.update_yaxes(title = 'Elevation Range (ft)' )
    fig.show()

## FILTERED P16

In [ ]:
filt_pct = df['Pass/Fail'] > 0.15
filt_STR = df['Threshold'] <= 10 
df_filt = df.loc[filt_pct & filt_STR].copy()

In [ ]:
df_filt.head()

In [ ]:
range(len(df['TargetThickness'].unique()))

In [ ]:
fig = make_subplots(rows=3, cols = 2,horizontal_spacing = 0.25,
                    vertical_spacing= 0.1, shared_xaxes = True)

rowCol = [(1,1),(1,2),(2,1),(2,2),(3,1),(3,2)]
for ii, pos in zip(np.sort(df['TargetThickness'].unique()), range(len(df['TargetThickness'].unique()))):
    
    dfPLOT = df_filt.loc[df['TargetThickness'] == ii]
    fig.add_trace(go.Heatmap(
                       z= dfPLOT['Pass/Fail']*100,
                       x= dfPLOT['Threshold'],
                       y= dfPLOT['ElevRange'], 
                       zmin = 0,
                       zmax = 100,
                       #colorscale = plotly.colors.diverging.Portland,
                       #colorbar_x = 1.05,
                        #colorbar_title = dict(text= 'Percentage of CPTs<br>Passing Criteria'),
                        #colorbar_ticklabelposition = 'outside bottom',
                        coloraxis = 'coloraxis'),
                        row = rowCol[pos][0],
                        col = rowCol[pos][1])


    fig.update_layout(template = 'simple_white',font = dict(family = 'Arial', color = 'black'),
                       title_text = 'Weak Layer Heatmap for TT = ' + str(ii),
                        xaxis5 = dict(title = 'Tip Resistance Threshold Value (tsf)'),
                        xaxis6 = dict(title = 'Tip Resistance Threshold Value (tsf)'))
    fig.update_coloraxes(colorscale = plotly.colors.diverging.Portland,
                             colorbar_x = 1.05,
                            colorbar_title = dict(text= 'Percentage of CPTs<br>Passing Criteria'),
                        colorbar_ticklabelposition = 'outside bottom')
     
    fig.update_xaxes(dtick =1 , range = [5,10])
    fig.update_yaxes(title = 'Elevation Range (ft)' )
fig.show()